# History, Data Analysis and CS Political Collaboration System

# Overview

This notebook implements a multi-agent collaboration system that combines historical research with data analysis and political science expert to answer complex historical questions. It leverages the power of large language models to simulate specialized agents working together to provide comprehensive answers.

# Setup

In [ ]:
# Install Modules for the RAG
!pip install beyondllm
!pip install llama-index-embeddings-gemini
!pip install llama-index-readers-web

In [ ]:
# Install Modules required for the Agent
!pip install -q -U langchain
!pip install --upgrade --quiet  langchain-google-genai
!pip install -q -U duckduckgo-search langchain-community
!pip install -q -U google-generativeai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.6 MB/s eta 0:00:00


In [ ]:
# Modules web search
!pip install -q -U google-search-results

  Preparing metadata (setup.py) ... done


In [ ]:
import os
import time

# Comment or uncomment according to installed modules

from langchain_google_genai import GoogleGenerativeAI, HarmBlockThreshold, HarmCategory
#from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.tools import DuckDuckGoSearchResults
#from langchain_community.tools import JinaSearch
#from langchain_community.tools.jina_search.tool import JinaSearch
from langchain_community.tools import TavilySearchResults
from langchain_community.utilities import SerpAPIWrapper
from typing import List, Dict
#from dotenv import load_dotenv

import google.generativeai as genai

#load_dotenv()
#os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

# Modules RAG
#from beyondllm import source,retrieve,embeddings,llms,generator
#from beyondllm.embeddings import GeminiEmbeddings

from rich.console import Console
from rich.markdown import Markdown

from google.colab import userdata

In [ ]:
# Config Gemini
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEYL')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
# Tavily Api Key
TAVILY_API_KEY=userdata.get('TAVILY_API_KEY')
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

In [ ]:
# SerpApiKey
SERPAPI_API_KEY= userdata.get('SERPAPI_API_KEY')
os.environ['SERPAPI_API_KEY'] = SERPAPI_API_KEY

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Gemini model list
print("List of models that support generateContent:\n")
for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
        print(m.name)

print("List of models that support embedContent:\n")
for m in genai.list_models():
    if "embedContent" in m.supported_generation_methods:
        print(m.name)

List of models that support generateContent:

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/learnlm-1.5-pro-experimental
models/gemini-exp-1114
models/gemini-exp-1121
models/gemini-exp-1206
List of models that support embedContent:

models/embedding-001
models/text-embedding-004


# LLM

In [ ]:
# Initialize OpenAI model
llm = ChatOpenAI(model="gpt-4o-mini", max_tokens=1000, temperature=0.7)

In [ ]:
# Initialize Gemini model
llm_g = GoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=GOOGLE_API_KEY,
    safety_settings={
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },generation_config=genai.types.GenerationConfig(
        # Only one candidate for now.
        #candidate_count=1,
        #stop_sequences=["x"],
        max_output_tokens=1500,
        temperature=0.8,
    ),

)


In [ ]:
# We test the response of the Gemini model from LangChain

response_agent_output = llm_g.invoke(
        "In the area of ​​AI, which is a neural network? Develop an example of a simple neural network in python code. Respond in Spanish."
    )

In [ ]:
print(response_agent_output)

En el área de la IA, una red neuronal es un modelo computacional inspirado en la estructura y funcionamiento del cerebro humano.  Está compuesta por nodos (neuronas) interconectados que procesan información.  Estas neuronas se organizan en capas: una capa de entrada que recibe datos, una o más capas ocultas que realizan transformaciones no lineales de los datos, y una capa de salida que produce el resultado.  La información fluye entre las capas a través de conexiones ponderadas (sinapsis), y el aprendizaje se produce ajustando estos pesos para minimizar la diferencia entre la salida predicha y la salida deseada.

A continuación, un ejemplo de una red neuronal simple en Python usando NumPy para la manipulación de matrices y la función sigmoide como función de activación:

```python
import numpy as np

# Función sigmoide
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

# Derivada de la función sigmoide
def sigmoid_derivative(x):
  return x * (1 - x)

# Estructura de la red neuronal (2 ent

In [ ]:
prompt_01 = """Eres un experto en Ciencias Políticas y busqueda web, que utiliza los siguientes parametros para la creación de un prompt de busqueda web:
Area = Ciencias Políticas
Palabras claves = Estado, Contemporáneo, caracteristicas, causas, economía, politica, tensiones, gobernanza, desafios, Inteligencia Artificial.
Sugerencias = Que es, cuales son las principales, cuales son las, describa las principales, como impacta, como influye.
Ejemplos = 5
Retorna los prompt de ejemplo con un separador de lineas (---).
"""
response_agent_output2 = llm_g.invoke(prompt_01)

In [ ]:
print(type(response_agent_output2))

<class 'str'>


In [ ]:
# Output display
print("-" * 100)  # Separator
Console().print(Markdown(response_agent_output2))

----------------------------------------------------------------------------------------------------


Aquí tienes 5 ejemplos de prompts de búsqueda web, optimizados para un experto en Ciencias Políticas, utilizando   
los parámetros especificados:                                                                                      

 1 "Características del Estado contemporáneo: análisis de las tensiones entre gobernanza, economía política y la   
   influencia de la Inteligencia Artificial"  Este prompt es amplio, abarcando varios aspectos clave y utilizando  
   un lenguaje académico.                                                                                          

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

 2 "¿Qué es el Estado contemporáneo? Describa las principales características y cómo la Inteligencia Artificial    
   impacta en su gobernanza y desafíos económicos."  Este prompt es más directo y utiliza las "sugerencias"        
   proporcionadas para guiar la búsqueda.                                                                          

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

 3 "Causas de las tensiones en el Estado contemporáneo: ¿Cómo influyen la economía política y el auge de la        
   Inteligencia Artificial en la gobernanza?" Este prompt se centra en las causas y las relaciones de causa-efecto,
   utilizando un lenguaje preciso.                                                                                 

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

 4 "Cuáles son las principales características del Estado contemporáneo en relación a la economía y la política?   
   Análisis de los desafíos y el rol de la Inteligencia Artificial."  Este prompt utiliza una estructura de        
   pregunta-respuesta y se enfoca en la relación entre economía, política y el Estado.                             

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

 5 "El Estado contemporáneo: ¿Cuáles son sus principales características y cómo impacta la Inteligencia Artificial 
   en sus tensiones internas y en la gobernanza global?" Este prompt es más amplio, considerando la perspectiva    
   global y la influencia de la IA en las tensiones internas y externas del Estado.

In [ ]:
print("-" * 100)  # Separator
Console().print(Markdown(response_agent_output2))

# Agent

## Class Agents

In [ ]:
# Define the base Agent class

class Agent:
    def __init__(self, name: str, role: str, skills: List[str]):
        self.name = name
        self.role = role
        self.skills = skills
        self.llm = llm_g

    def process(self, task: str, context: List[Dict] = None) -> str:
        messages = [
            SystemMessage(content=f"You are {self.name}, a {self.role}. Your skills include: {', '.join(self.skills)}. Respond to the task based on your role and skills.")
        ]

        if context:
            for msg in context:
                if msg['role'] == 'human':
                    messages.append(HumanMessage(content=msg['content']))
                elif msg['role'] == 'ai':
                    messages.append(AIMessage(content=msg['content']))

        messages.append(HumanMessage(content=task))
        response = self.llm.invoke(messages)
        return response# .content

Define specialized agents:

* HistoryResearchAgent
* DataAnalysisAgent
* PoliticalScienceExpertAgent

In [ ]:

class HistoryResearchAgent(Agent):
    def __init__(self):
        super().__init__("Clio", "History Research Specialist", ["deep knowledge of historical events", "understanding of historical contexts", "identifying historical trends"])

class PoliticalScienceExpertAgent(Agent):
    def __init__(self):
        super().__init__("Politic", "Political Science Expert", ["deep knowledge of political science", "understanding of political contexts", "senior academic level in political science"])

class DataAnalysisAgent(Agent):
    def __init__(self):
        super().__init__("Data", "Data Analysis Expert", ["interpreting context data", "deep data analysis", "description of analyzed data"])

## RAG

In [ ]:
# You can use BeyondLLM

# query_p = "Según la ciencia política, desarrollar y explicar los principales acontecimientos que provocaron el tránsito del Estado moderno al Estado contemporáneo, sus características y tensiones de esta etapa."
# data_pdf = source.fit(path=".pdf", dtype="pdf", chunk_size=1024, chunk_overlap=100)
# embed_model_pdf = GeminiEmbeddings(api_key=GOOGLE_API_KEY, model_name="models/embedding-001")
# retriever_pdf = retrieve.auto_retriever(data=data_pdf, embed_model=embed_model_pdf, type="hybrid", mode="OR", top_k=3)
# llm_rag = llms.GeminiModel(model_name="gemini-pro", google_api_key=GOOGLE_API_KEY)
# pipeline = generator.Generate(question=query_p, retriever=retriever_pdf, llm=llm)
# ragcspolitical_response = pipeline.call()

## Web search

In [ ]:
search_p = DuckDuckGoSearchRun()

# DuckDuckGoSearchResults() -> retorna un str
search_dduckgo = DuckDuckGoSearchResults(output_format="list", num_results=3) # -> retorna una lista

# "Develop and fully explain what the contemporary State is and its main characteristics."
# "Transition from the modern State to the contemporary State."

response_searchp = search_dduckgo.invoke("Transition from the modern State to the contemporary State.")

In [ ]:
print(type(response_searchp))

<class 'list'>


In [ ]:
print(response_searchp)

[{'snippet': "Enduring Legacies and Historical Disjunctures' aims to critically assess arguably one of the most important historical developments in international relations (IR) during the modern period - namely, the transition from empires to nation states. This transition had fundamental consequences in the development of International Relations in the ...", 'title': 'From Empires to Nation States? Enduring Legacies and Historical ...', 'link': 'https://journals.sagepub.com/doi/10.1177/03058298231214004'}, {'snippet': "However, in Quijano's formulation, this democratisation is seen as a necessary condition to the establishment of the modern nation-state: 'only through the process of democratization of society can the construction of a modern nation-state, with all of its implications, including citizenship and political representation, be possible and ...", 'title': 'Against Sovereignty: The Colonial Limits of Modern Politics', 'link': 'https://journals.sagepub.com/doi/full/10.1177/0

In [ ]:
print(type(response_searchp))

<class 'list'>


In [ ]:
# response_searchp [0]['link']
print(response_searchp[0]['link'])

https://journals.sagepub.com/doi/10.1177/03058298231214004


In [ ]:
output_search_list = [i['link']for i in response_searchp]

In [ ]:
print(output_search_list)

['https://journals.sagepub.com/doi/10.1177/03058298231214004', 'https://journals.sagepub.com/doi/full/10.1177/03058298231194742', 'https://www.cambridge.org/core/journals/social-science-history/article/reverberations-of-empire-how-the-colonial-past-shapes-the-present/178FA24536F578B3EFE2434DFDB87846']


In [ ]:
print("-" * 100)  # Separator
Console().print(Markdown(output_search_list[0]))
print(output_search_list[0])
#Console().print(Markdown(response_agent_output2))

----------------------------------------------------------------------------------------------------


https://journals.sagepub.com/doi/10.1177/03058298231214004

https://journals.sagepub.com/doi/10.1177/03058298231214004


In [ ]:
urls_ddgo = output_search_list
data_rag = source.fit(path=urls_ddgo, dtype="url")
query_rag = "Transition from the modern State to the contemporary State."
embed_model_rag = GeminiEmbeddings(api_key=GOOGLE_API_KEY, model_name="models/embedding-001")
retriever_pdf = retrieve.auto_retriever(data=data_rag, embed_model=embed_model_rag, type="hybrid", mode="OR", top_k=3)
llm_rag = llms.GeminiModel(model_name="gemini-pro", google_api_key=GOOGLE_API_KEY)
pipeline = generator.Generate(question=query_rag, retriever=retriever_pdf, llm=llm)
ragcspolitical_response = pipeline.call()

In [ ]:
# Tool Tavily Search

search_t = TavilySearchResults(
    max_results=3,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=False,
    # include_domains=[...],
    # exclude_domains=[...],
    # name="...",            # overwrite default tool name
    # description="...",     # overwrite default tool description
    # args_schema=...,       # overwrite default args_schema: BaseModel
)

In [ ]:
response_searcht = search_t.invoke({"query": "According to political science, what is the contemporary State, mention its main economic causes."})

In [ ]:
print(response_searcht)

[{'url': 'https://www.cambridge.org/core/books/introduction-to-comparative-politics/modern-state/47BB2E874F4D3C839425A1E15ECEAEC0', 'content': 'Summary. This book is about states and the challenges to sovereignty they face in the contemporary world. We address this issue by systematically comparing states around the globe. As such, this book represents a contribution to comparative politics, which is a core subfield within the academic discipline of political science.'}, {'url': 'https://www.britannica.com/topic/political-science', 'content': 'Political philosophy is concerned primarily with political ideas and values, such as rights, justice, freedom, and political obligation (whether people should or should not obey political authority); it is normative in its approach (i.e., it is concerned with what ought to be rather than with what is) and rationalistic in its method. Although political science borrows heavily from the other social sciences, it is distinguished from them by its fo

In [ ]:
content_t = ""
urls_t = ""
for i in response_searcht:
    urls_t += f"{i['url']}\n\n"
    content_t += f"{i['content']}\n\n"
    print(i['content'])

Summary. This book is about states and the challenges to sovereignty they face in the contemporary world. We address this issue by systematically comparing states around the globe. As such, this book represents a contribution to comparative politics, which is a core subfield within the academic discipline of political science.
Political philosophy is concerned primarily with political ideas and values, such as rights, justice, freedom, and political obligation (whether people should or should not obey political authority); it is normative in its approach (i.e., it is concerned with what ought to be rather than with what is) and rationalistic in its method. Although political science borrows heavily from the other social sciences, it is distinguished from them by its focus on power—defined as the ability of one political actor to get another actor to do what it wants—at the international, national, and local levels. However, if the term science applies to any body of systematically orga

In [ ]:
print(content_t)
print("*"*100)
print(urls_t)

Summary. This book is about states and the challenges to sovereignty they face in the contemporary world. We address this issue by systematically comparing states around the globe. As such, this book represents a contribution to comparative politics, which is a core subfield within the academic discipline of political science.

Political philosophy is concerned primarily with political ideas and values, such as rights, justice, freedom, and political obligation (whether people should or should not obey political authority); it is normative in its approach (i.e., it is concerned with what ought to be rather than with what is) and rationalistic in its method. Although political science borrows heavily from the other social sciences, it is distinguished from them by its focus on power—defined as the ability of one political actor to get another actor to do what it wants—at the international, national, and local levels. However, if the term science applies to any body of systematically org

In [ ]:
# print(response_searcht[0]['content'])
print("*"*100)
print("-" * 100)  # Separator
Console().print(Markdown(content_t))
print("-" * 100)  # Separator
print(urls_t)

****************************************************************************************************
----------------------------------------------------------------------------------------------------


Summary. This book is about states and the challenges to sovereignty they face in the contemporary world. We       
address this issue by systematically comparing states around the globe. As such, this book represents a            
contribution to comparative politics, which is a core subfield within the academic discipline of political science.

Political philosophy is concerned primarily with political ideas and values, such as rights, justice, freedom, and 
political obligation (whether people should or should not obey political authority); it is normative in its        
approach (i.e., it is concerned with what ought to be rather than with what is) and rationalistic in its method.   
Although political science borrows heavily from the other social sciences, it is distinguished from them by its    
focus on power—defined as the ability of one political actor to get another actor to do what it wants—at the       
international, national, and local levels. However, if the term science applies to any body of systematically      
organized knowledge based on facts ascertained by empirical methods and described by as much measurement as the    
material allows, then political science is a science, like the other social disciplines. Political science is      
generally used in the singular, but in French and Spanish the plural (sciences politiques and ciencias políticas,  
respectively) is used, perhaps a reflection of the discipline’s eclectic nature. The contemporary discipline,      
however, is considerably broader than this, encompassing studies of all the societal, cultural, and psychological  
factors that mutually influence the operation of government and the body politic.                                  

Constructing political institutions where none existed previously or were very weak. Failed-states. States that    
have no effective central control, no monopoly on the use of violence, porous borders. Study with Quizlet and      
memorize flashcards containing terms like The Modern State, Development of Modern State, First State and more.

----------------------------------------------------------------------------------------------------
https://www.cambridge.org/core/books/introduction-to-comparative-politics/modern-state/47BB2E874F4D3C839425A1E15ECEAEC0

https://www.britannica.com/topic/political-science

https://quizlet.com/173185150/political-science-the-modern-state-flash-cards/




In [ ]:
# Search SerpAPIWrapper

# engine: "bing"
params_w = {
    "engine": "google",
    "gl": "us",
    "hl": "en",
}
search_w = SerpAPIWrapper(params=params_w)
response_searchw = search_w.run("Economic causes of the transition from the modern State to the contemporary State.")

In [ ]:
print((response_searchw))

['The transition process is usually characterized by the changing and creating of institutions, particularly private enterprises; changes in the role of the state ...', 'Drive to Maturity: This stage takes place over a long period of time, as standards of living rise, use of technology increases, and the national economy grows ...', 'In some historical periods the immediate formative influence of the fiscal needs and policy of the state on the development of the economy and with it on all ...', 'Factors such as internal political stability and conflict, political conflict between competing states, culture and society within a state, ...', 'The most prevalent argument has focused on the effects of globalization. Increasing trade and financial flows are said to put pressure on national economies.', 'Repeated foreign invasions from the Hyksos in 1650 BCE and onwards implied that Egypt gradually lost its economic and military dominance over ...', 'Industrialization is the process of transf

In [ ]:
print("-" * 100)  # Separator
Console().print(Markdown(response_searchw))
print("-" * 100)  # Separator

----------------------------------------------------------------------------------------------------


['The transition process is usually characterized by the changing and creating of institutions, particularly       
private enterprises; changes in the role of the state ...', 'Drive to Maturity: This stage takes place over a long 
period of time, as standards of living rise, use of technology increases, and the national economy grows ...', 'In 
some historical periods the immediate formative influence of the fiscal needs and policy of the state on the       
development of the economy and with it on all ...', 'Factors such as internal political stability and conflict,    
political conflict between competing states, culture and society within a state, ...', 'The most prevalent argument
has focused on the effects of globalization. Increasing trade and financial flows are said to put pressure on      
national economies.', 'Repeated foreign invasions from the Hyksos in 1650 BCE and onwards implied that Egypt       
gradually lost its economic and military dominance over ...', 'Industrialization is the process of transforming the
economy of a nation or region from a focus on agriculture to a reliance on manufacturing.', 'A transitional society
refers to a country undergoing macroeconomic reforms to shift from a state-run economy to a more market-led        
system.', 'New economic opportunities are limited by high barriers to entry and growth, such as bureaucratic       
interference, corruption, and crime, or by lack of access to ...', 'This economy can therefore be represented as a 
repeated game between the elite and the poor. We characterize the pure strategy Markov perfect equilibria of this  
...']

----------------------------------------------------------------------------------------------------


## Tools

Define the different functions for the collaboration system

In [ ]:
# RAG of political science concepts

#def rag_cspolitical_context(cspolitical_agent, task: str, context: list) -> list:
    #print("📚 CS Political Agent: RAG Researching and analyzing political science concepts...")
    #research_cspolitical = context[-1]["content"]
    #ragcspolitical_task = f"Provide relevant context and information for the following task: {task}"
    #ragcspolitical_task = f"Analyze the data provided and describe any trends or insights relevant to the original task. Analyze data: {research_cspolitical}}"
    #data_pdf = source.fit(path=".pdf", dtype="pdf", chunk_size=1024, chunk_overlap=100)
    #embed_model_pdf = GeminiEmbeddings(api_key=GOOGLE_API_KEY, model_name="models/embedding-001")
    #retriever_pdf = retrieve.auto_retriever(data=data_pdf, embed_model=embed_model_pdf, type="hybrid", mode="OR", top_k=3)
    #llm = llms.GeminiModel(model_name="gemini-pro", google_api_key=GOOGLE_API_KEY)
    #pipeline = generator.Generate(question=ragcspolitical_task, retriever=retriever_pdf, llm=llm)
    #ragcspolitical_response = pipeline.call()
    #ragcspolitical_result = cspolitical_agent.process(ragcspolitical_response, context)
    #context.append({"role": "ai", "content": f"CS Political Agent: {ragcspolitical_result}"})
    #print(f"📋 CS Political RAG results: {ragcspolitical_result[:100]}...\n")
    #return context

In [ ]:
# Web research and analysis of political science concepts / Tool -> DuckDuckGoSearchRun()

def webresearch_dduckgo(task: str):
    research = DuckDuckGoSearchRun()
    response = research.invoke(task)
    return response


def webresearch_cspolitical_context(cspolitical_agent, task: str, context: list) -> list:
    print("🕵️ CS Political Agent: Web researching for political science concepts...")
    #rag_cspolitical = context[-1]["content"]
    #webresearch_csp_task = f"Provide relevant context and information for the following task: {task}"
    webresearch_csp_task = f"{task}"
    #webresearch_csp_task = f"Analyze the data provided and describe any trends or insights relevant to the original task. Analyze data: {task}"
    #webresearch_csp_result = cspolitical_agent.process(cspolitical_task, context)
    webresearch_csp_response = webresearch_dduckgo(webresearch_csp_task)
    #webresearch_csp_response = web_research.invoke(webresearch_csp_task)
    webresearch_csp_result = cspolitical_agent.process(webresearch_csp_task)
    context.append({"role": "ai", "content": f"CS Political Agent: {webresearch_csp_response}"})
    print(f"📋 CS Political web research results: {webresearch_csp_response[:100]}...\n")
    return context

In [ ]:
# Research Historical Context

def research_historical_context(history_agent, task: str, context: list) -> list:
    print("🏛️ History Agent: Researching historical context...")
    #ragcspolitical_context = context[-1]["content"]
    #webresearch_csp = context[-1]["content"]
    #history_task = f"Based on the web research context, what specific data or statistical information would be helpful to answer the original question? Web research context: {webresearch_csp}"
    history_task = f"Provide relevant historical context and information for the following task: {task}"
    history_result = history_agent.process(history_task)
    context.append({"role": "ai", "content": f"History Agent: {history_result}"})
    print(f"📜 Historical context provided: {history_result[:100]}...\n")
    return context

In [ ]:
# Identify Data Needs

def identify_data_needs(data_agent, task: str, context: list) -> list:
    print("📊 Data Agent: Identifying data needs based on historical context...")
    historical_context = context[-1]["content"]
    data_need_task = f"Based on the historical context, what specific data would be helpful to answer the original task. Historical context: {historical_context}"
    data_need_result = data_agent.process(data_need_task, context)
    context.append({"role": "ai", "content": f"Data Agent: {data_need_result}"})
    print(f"🔍 Data identified: {data_need_result[:100]}...\n")
    return context

In [ ]:
# Provide Historical Data

def provide_historical_data(history_agent, task: str, context: list) -> list:
    print("🏛️ History Agent: Providing relevant historical data...")
    data_needs = context[-1]["content"]
    data_provision_task = f"Based on the data identified, provide relevant historical data. Data needs: {data_needs}"
    data_provision_result = history_agent.process(data_provision_task, context)
    context.append({"role": "ai", "content": f"History Agent: {data_provision_result}"})
    print(f"📊 Historical data provided: {data_provision_result[:100]}...\n")
    return context

In [ ]:
# Analyze Data

def analyze_data(data_agent, task: str, context: list) -> list:
    print("📈 Data Agent: Analyzing historical data...")
    historical_data = context[-1]["content"]
    analysis_task = f"Analyze the historical data provided and describe any trends or insights relevant to the original task. Historical data: {historical_data}"
    analysis_result = data_agent.process(analysis_task, context)
    context.append({"role": "ai", "content": f"Data Agent: {analysis_result}"})
    print(f"💡 Data analysis results: {analysis_result[:100]}...\n")
    return context

In [ ]:
# Research and analysis of political science concepts

def research_cspolitical_context(cspolitical_agent, task: str, context: list) -> list:
    print("📚 CS Political Agent: Searching and analyzing political science concepts...")
    analyze_data = context[-1]["content"]
    cspolitical_task = f"Analyze the data provided as a political science expert and describe any trends or insights relevant to the original task. Analyze data: {analyze_data}"
    cspolitical_result = cspolitical_agent.process(cspolitical_task, context)
    context.append({"role": "ai", "content": f"CS Political Agent: {cspolitical_result}"})
    print(f"📋 CS Political research and analysis results: {cspolitical_result[:100]}...\n")
    return context

In [ ]:
# Synthesize Final Answer

def synthesize_final_answer(cspolitical_agent, task: str, context: list) -> str:
    print("🏛️ Multi Agent: Synthesizing final answer...")
    synthesis_task = "Based on all the historical context, data, and analysis, provide a comprehensive answer to the original task. Response in spanish."
    final_result = cspolitical_agent.process(synthesis_task, context)
    print("\n✅ Collaboration complete.\n")
    return final_result

HistoryDataCollaborationSystem Class

In [ ]:
class HistoryDataCollaborationSystem:
    def __init__(self):
        self.history_agent = HistoryResearchAgent() #Agent("Clio", "History Research Specialist", ["deep knowledge of historical events", "understanding of historical contexts", "identifying historical trends"])
        self.data_agent = DataAnalysisAgent() #Agent("Data", "Data Analysis Expert", ["interpreting numerical data", "statistical analysis", "data visualization description"])
        self.cspolitical_agent = PoliticalScienceExpertAgent()

    def solve(self, task: str, timeout: int = 300) -> str:
        print(f"\n👥 Starting collaboration to solve: {task}\n")

        start_time = time.time()
        context = []

        steps = [
            (research_historical_context, self.history_agent),
            (identify_data_needs, self.data_agent),
            (provide_historical_data, self.history_agent),
            (analyze_data, self.data_agent),
            (research_cspolitical_context, self.cspolitical_agent),
            (synthesize_final_answer, self.cspolitical_agent)
        ]

        for step_func, agent in steps:
            if time.time() - start_time > timeout:
                return "Operation timed out. The process took too long to complete."
            try:
                result = step_func(agent, task, context)
                if isinstance(result, str):
                    return result  # This is the final answer
                context = result
            except Exception as e:
                return f"Error during collaboration: {str(e)}"

        #print("\n✅ Collaboration complete. Final answer synthesized.\n")
        #return "hola mundo"#context[-1]["content"]
        #print(f"Muestro contexto: {context}")

In [ ]:
print("*"*100)

****************************************************************************************************


In [ ]:
steps2 = [
            ("Funcion 1", "history_agent"),
            ("Funcion 2", "data_agent2"),
            ("Funcion 3", "history_agent2"),
            ("Funcion 4", "data_agent2"),
            ("Funcion 5", "history_agent3")
        ]

for step_func2, agent2 in steps2:
    print(step_func2, agent2)

Funcion 1 history_agent
Funcion 2 data_agent2
Funcion 3 history_agent2
Funcion 4 data_agent2
Funcion 5 history_agent3


# Inference

Example usage

In [ ]:
# Prompt generator
prompt_02 = """Eres un experto en Ciencias Políticas y busqueda web, que utiliza los siguientes parametros para la creación de un prompt de busqueda web:
Area = Ciencias Políticas
Palabras claves = Estado, Contemporáneo, caracteristicas, causas, economía, politica, tensiones, gobernanza, desafios, Inteligencia Artificial.
Sugerencias = Que es, cuales son las principales, cuales son las, describa las principales, como impacta, como influye.
Ejemplos = 1
Retorna el prompt de ejemplo en inglés seguido de "Task: "
"""
response_generator = llm_g.invoke(prompt_02)

In [ ]:
# Create an instance of the collaboration system
collaboration_system = HistoryDataCollaborationSystem()

# Define a complex historical question that requires both historical knowledge and data analysis

# questions = ["How did urbanization rates in Europe compare to those in North America during the Industrial Revolution, and what were the main factors influencing these trends?",
# "What are the most important economic factors that caused the industrial revolution?",
# "According to political science, what are the main economic causes of the modern state and its main thinkers?",
# "According to political science, the modern state gave way to the contemporary state, develop the main events that triggered this change and explain the development and tensions that characterized this stage.",
# "According to political science, develop and explain the main events that caused the transition from the modern state to the contemporary state, its characteristics and tensions of this stage."]
task_p1 = "What are the main characteristics of the contemporary State and how does Artificial Intelligence influence in its governance?"
question = task_p1

# Solve the question using the collaboration system
result = collaboration_system.solve(question)

# Print the result
#print(result)


👥 Starting collaboration to solve: What are the main characteristics of the contemporary State and how does Artificial Intelligence influence in its governance?

🏛️ History Agent: Researching historical context...
📜 Historical context provided: The contemporary State, in its broadest definition, is a complex entity significantly different from...

📊 Data Agent: Identifying data needs based on historical context...
🔍 Data identified: To effectively analyze the influence of AI on state governance within its historical context, severa...

🏛️ History Agent: Providing relevant historical data...
📊 Historical data provided: I cannot directly provide the *data* requested.  My capabilities are limited to processing and gener...

📈 Data Agent: Analyzing historical data...
💡 Data analysis results: The provided text outlines a framework for analyzing the impact of AI on state governance, but lacks...

📚 CS Political Agent: Searching and analyzing political science concepts...
📋 CS Political rese

In [ ]:
print("-" * 100)  # Separator
Console().print(Markdown(result))

----------------------------------------------------------------------------------------------------


La influencia de la inteligencia artificial (IA) en la gobernanza estatal contemporánea es un tema complejo que    
requiere un análisis multifacético, considerando tanto su continuidad con las tendencias históricas como sus       
desafíos novedosos.  El Estado moderno, producto de siglos de evolución, se ha visto transformado por la emergencia
de los estados-nación, el estado de bienestar, la globalización y, ahora, la revolución digital. La IA actúa como  
un catalizador, amplificando tendencias existentes y creando nuevas dinámicas.                                     

Continuidad y Cambio: La búsqueda del Estado por el control y la eficiencia es una constante histórica, desde las  
monarquías centralizadas hasta el estado de bienestar.  La IA proporciona herramientas para optimizar estos        
objetivos: la vigilancia masiva, la toma de decisiones algorítmicas, la automatización de procesos burocráticos.   
Sin embargo, la escala y la velocidad a la que opera la IA superan cualquier precedente histórico.  La vigilancia  
algorítmica, por ejemplo, difiere de las formas históricas de control por su alcance y su potencial para el sesgo y
la discriminación.  El análisis comparativo de la efectividad de métodos históricos de control con las capacidades 
de la IA es crucial.                                                                                               

Relación Estado-Ciudadano: El estado de bienestar representó un cambio hacia una mayor intervención del Estado en  
el bienestar ciudadano, reflejando un pacto social implícito. La IA puede fortalecer o debilitar este pacto.  Si   
bien puede mejorar la eficiencia en la prestación de servicios públicos, el sesgo algorítmico en la asignación de  
recursos puede exacerbar las desigualdades, erosionando la confianza ciudadana.  Es necesario comparar los impactos
de los programas de bienestar históricos con las iniciativas impulsadas por la IA, midiendo sus efectos en         
indicadores sociales como la pobreza y el acceso a los servicios.  La transparencia y la rendición de cuentas en   
los sistemas algorítmicos son fundamentales para mantener la legitimidad del Estado.                               

Globalización y Soberanía: La globalización ha desafiado históricamente la soberanía nacional.  La IA intensifica  
este desafío a través de las amenazas cibernéticas, la desinformación y las armas autónomas.  El análisis          
comparativo de avances tecnológicos anteriores (telégrafo, internet) con la IA es crucial para entender su impacto 
en la soberanía.  La cooperación internacional en el desarrollo y la regulación de la IA será determinante para    
evitar una mayor fragmentación y conflicto entre Estados.                                                          

Cambio Tecnológico y Desigualdad:  El progreso tecnológico ha generado históricamente tanto avances como           
desigualdades.  La IA, con su potencial de automatización, plantea preocupaciones similares sobre el desplazamiento
laboral y el acceso desigual a sus beneficios.  Un análisis exhaustivo debe considerar los impactos directos e     
indirectos de la IA en la distribución de la riqueza, el acceso a la educación y la salud, y la participación      
política.  La comparación con transiciones tecnológicas anteriores debe tener en cuenta la velocidad y la escala   
del cambio actual.                                                                                                 

Transparencia y Rendición de Cuentas:  La transparencia y la rendición de cuentas han sido históricamente procesos 
lentos e irregulares. La IA plantea un reto único debido a la opacidad de muchos algoritmos, dificultando la       
comprensión de los procesos de toma de decisiones y la rendición de cuentas.  Es necesario desarrollar mecanismos  
innovadores para la transparencia algorítmica y la rendición de cuentas, incluyendo nuevos marcos legales y        
mecanismos de supervisión independientes.          

In [ ]:
collaboration_system2 = HistoryDataCollaborationSystem()

task_p2 = "Describe the main political and economic tensions in the contemporary State caused by development and the implementation of Artificial Intelligence."
question2 = task_p2

# Solve the question using the collaboration system
result2 = collaboration_system2.solve(question2)


👥 Starting collaboration to solve: Describe the main political and economic tensions in the contemporary State caused by development and the implementation of Artificial Intelligence.

🏛️ History Agent: Researching historical context...
📜 Historical context provided: The contemporary state faces significant political and economic tensions stemming from AI developmen...

📊 Data Agent: Identifying data needs based on historical context...
🔍 Data identified: To effectively analyze the contemporary tensions surrounding AI development and implementation, and ...

🏛️ History Agent: Providing relevant historical data...
📊 Historical data provided: Unfortunately, providing the precise quantitative data requested from the historical periods mention...

📈 Data Agent: Analyzing historical data...
💡 Data analysis results: Analysis of Historical Data and Trends Relevant to AI Tensions

The provided historical data, while ...

📚 CS Political Agent: Searching and analyzing political science concepts

In [ ]:
print("-" * 100)  # Separator
Console().print(Markdown(result2))

----------------------------------------------------------------------------------------------------


Las tensiones políticas y económicas contemporáneas derivadas del desarrollo e implementación de la IA hacen eco de
las ansiedades históricas en torno a los avances tecnológicos.  Como experto en ciencia política, puedo afirmar que
comprender estas tensiones requiere un análisis que integre la perspectiva histórica con el contexto actual.       

Paralelos históricos:                                                                                              

El análisis de la Historia Agent identifica correctamente paralelos cruciales: el movimiento ludita, la Segunda    
Revolución Industrial, la carrera armamentística de la Guerra Fría y el auge de internet y la globalización.  Cada 
uno de estos periodos evidenció tensiones similares a las actuales: desplazamiento laboral, desigualdad económica, 
concentración del poder y preocupaciones éticas.  Sin embargo, la escala, velocidad y complejidad del desarrollo de
la IA presentan desafíos únicos.                                                                                   

Análisis de la Data Agent:                                                                                         

La Data Agent destaca acertadamente la necesidad de datos cuantitativos y cualitativos tanto históricos como       
contemporáneos para una comprensión completa.  Si bien la recopilación de datos precisos de periodos históricos es 
difícil, la Data Agent proporciona valiosas sugerencias para la investigación en archivos y bases de datos         
especializados.  La identificación de tendencias recurrentes –desplazamiento tecnológico, concentración del poder, 
preocupaciones éticas e imprevisibilidad– es crucial.  La ausencia de datos cuantitativos históricos precisos      
subraya la necesidad de una recolección de datos rigurosa en el presente para informar las decisiones políticas    
futuras.                                                                                                           

Perspectiva de la Ciencia Política (Politic):                                                                      

La perspectiva de la ciencia política añade una capa crucial al análisis.  No se trata solo de identificar patrones
históricos, sino de comprender los mecanismos políticos que moldean la respuesta a estos desafíos.  La voluntad    
política para implementar estrategias de mitigación (redes de seguridad social, programas de reciclaje profesional)
será fundamental para determinar el impacto social del desplazamiento laboral impulsado por la IA.  La falta de    
esta voluntad política podría provocar inestabilidad social y política.                                            

La concentración del poder en manos de unas pocas empresas tecnológicas plantea preocupaciones sobre la regulación 
del mercado, las leyes antimonopolio y la influencia indebida en los procesos políticos.  La ciencia política      
proporciona marcos para analizar las dinámicas de poder, el cabildeo y la captura regulatoria, elementos cruciales 
para comprender la influencia de estas empresas.                                                                   

Las preocupaciones éticas son intrínsecamente políticas.  El debate sobre las armas autónomas, por ejemplo,        
involucra la naturaleza de la guerra, el derecho internacional y el papel de la agencia humana.  La ciencia        
política ayuda a comprender cómo diferentes actores enmarcan estos debates y cómo estos marcos influyen en los     
resultados políticos.                                                                                              

La imprevisibilidad del cambio tecnológico exige estructuras de gobernanza adaptativas.  Una regulación rígida     
puede sofocar la innovación, mientras que la ausencia de regulación puede llevar a consecuencias catastróficas. La 
ciencia política ofrece modelos para la toma de decisiones en condiciones de incertidumbre, incluyendo mecanismos  
para ajustes iterativos de políticas, participa

In [ ]:
# Prompt generator
prompt_02 = """Eres un experto en Ciencias Políticas y busqueda web, que utiliza los siguientes parametros para la creación de un prompt de busqueda web:
Area = Ciencias Políticas
Palabras claves = Estado, Contemporáneo, caracteristicas, causas, economía, politica, tensiones, gobernanza, desafios, Inteligencia Artificial.
Sugerencias = Que es, cuales son las principales, cuales son las, describa las principales, como impacta, como influye.
Ejemplos = 1
Retorna el prompt de ejemplo en inglés dentro de:" "
"""
response_generator = llm_g.invoke(prompt_02)

In [ ]:
print(response_generator)

"What are the main characteristics of the contemporary state, its causes, and its relationship with economics and politics?  How do tensions between governance challenges and the impact of Artificial Intelligence influence the contemporary state?"



In [ ]:
collaboration_system3 = HistoryDataCollaborationSystem()

#task_p2 = "Describe the main political and economic tensions in the contemporary State caused by development and the implementation of Artificial Intelligence."
question3 = response_generator

# Solve the question using the collaboration system
result3 = collaboration_system3.solve(question3)


👥 Starting collaboration to solve: "What are the main characteristics of the contemporary state, its causes, and its relationship with economics and politics?  How do tensions between governance challenges and the impact of Artificial Intelligence influence the contemporary state?"


🏛️ History Agent: Researching historical context...
📜 Historical context provided: The contemporary state, a complex entity vastly different from its predecessors, is characterized by...

📊 Data Agent: Identifying data needs based on historical context...
🔍 Data identified: To effectively analyze the contemporary state and its relationship with AI, based on the provided hi...

🏛️ History Agent: Providing relevant historical data...
📊 Historical data provided: As Clio, the History Research Specialist, I cannot directly provide the specific quantitative data r...

📈 Data Agent: Analyzing historical data...
💡 Data analysis results: The provided historical overview lacks the quantitative data necessary for ro

In [ ]:
print("-" * 100)  # Separator
Console().print(Markdown(result3))

----------------------------------------------------------------------------------------------------


Basado en el contexto histórico, los datos y el análisis proporcionados, podemos concluir que el Estado            
contemporáneo es una entidad compleja y dinámica, radicalmente diferente de sus predecesores, caracterizada por una
serie de rasgos interconectados que requieren un análisis multifacético.  No es posible ofrecer una respuesta      
definitiva sin datos cuantitativos exhaustivos, pero podemos esbozar una comprensión general, destacando las       
limitaciones de la información disponible.                                                                         

Características Principales del Estado Contemporáneo:                                                              

El Estado contemporáneo se caracteriza por una mayor capacidad de intervención en la sociedad, aunque con          
variaciones significativas entre países. Esto se refleja en burocracias más sofisticadas, sistemas tributarios más 
complejos y una mayor capacidad para hacer cumplir las leyes.  La globalización e interdependencia han generado una
complejidad adicional, desafiando la soberanía estatal tradicional a través de organizaciones supranacionales y la 
cooperación internacional en temas cruciales.  La provisión del Estado de bienestar, aunque variable en su alcance,
es otro rasgo distintivo, con un impacto significativo en la vida social y económica de los ciudadanos.  La mayoría
de los Estados contemporáneos se presentan como democráticos (o parcialmente democráticos), aunque la calidad de la
democracia varía considerablemente, con desafíos como la corrupción y la polarización. Finalmente, la dependencia  
tecnológica, especialmente la influencia de la IA, redefine las capacidades estatales, presentando tanto           
oportunidades (mejora de la eficiencia, análisis de datos) como riesgos (sesgos algorítmicos, vigilancia masiva,   
desplazamiento laboral).                                                                                           

Causas Históricas:                                                                                                 

La configuración del Estado contemporáneo es el resultado de un proceso histórico complejo.  La Paz de Westfalia   
(1648) sentó las bases del sistema estatal moderno, estableciendo el principio de soberanía.  El auge del          
nacionalismo en los siglos XIX y XX impulsó la formación de nuevos Estados-nación.  La industrialización y el      
capitalismo transformaron las estructuras económicas, requiriendo una mayor intervención estatal.  Las guerras     
mundiales y la descolonización reconfiguraron el mapa político global, dando lugar a numerosos nuevos Estados.     

La Relación con la Economía y la Política:                                                                         

La interdependencia entre el Estado, la economía y la política es innegable. Las políticas económicas del Estado   
moldean el panorama social y político, mientras que el sistema político influye en el desempeño económico. La      
teoría económica keynesiana, por ejemplo, tuvo un impacto profundo en el papel del Estado en la gestión de la      
economía después de la Segunda Guerra Mundial.                                                                     

El Desafío de la IA:                                                                                               

La IA presenta una doble cara para el Estado.  Puede mejorar la capacidad estatal, pero también plantea desafíos   
éticos y políticos significativos.  El sesgo algorítmico, la privacidad de los datos, el desplazamiento laboral y  
el desarrollo de sistemas de armas autónomas son preocupaciones centrales que requieren una regulación cuidadosa y 
una consideración ética profunda.                                                                                  

Limitaciones del Análisis:                                                                                         

Es crucial enfatizar que el análisis anterior 

# Resources

* [API de Gemini](https://ai.google.dev/gemini-api/docs?hl=es-419)
* [LangChain](https://python.langchain.com/docs/introduction/)
* [BeyondLLM](https://beyondllm.aiplanet.com/)